## Kruskal's Algorithm

In [1]:
import networkx as nx
from math import inf

def kruskal(G, w):
    edges = [(u, v, w[u][v]) for u in G for v in G[u] if u < v and w[u][v] != float(inf)]
    edges.sort(key=lambda x: x[2])
    mst = nx.Graph()
    mst.add_nodes_from(G)
    for u, v, weight in edges:
        if not nx.has_path(mst, u, v):
            mst.add_edge(u, v, weight=weight)
    return mst
G = {
    "G": {"B": 5, "A": 4},
    "B": {"G": 5, "A": 1, "E": 2, "D": 2},
    "A": {"G": 4, "B": 1, "D": 4},
    "D": {"A": 4, "B": 2, "E": 3, "F": 5},
    "E": {"B": 2, "D": 3, "F": 1, "C": 4},
    "F": {"D": 5, "E": 1, "C": 3},
    "C": {"E": 4, "F": 3}
}
w = G 
MST = kruskal(G, w)
print("Minimum Spanning Tree Edges:", MST.edges(data=True))

Minimum Spanning Tree Edges: [('G', 'A', {'weight': 4}), ('B', 'A', {'weight': 1}), ('B', 'E', {'weight': 2}), ('B', 'D', {'weight': 2}), ('E', 'F', {'weight': 1}), ('F', 'C', {'weight': 3})]


## Kruskal's Algorithm validation with another graph

In [2]:
import networkx as nx
from math import inf

def kruskal(G, w):
    edges = [(u, v, w[u][v]) for u in G for v in G[u] if u < v and w[u][v] != float(inf)]
    edges.sort(key=lambda x: x[2])
    mst = nx.Graph()
    mst.add_nodes_from(G)
    for u, v, weight in edges:
        if not nx.has_path(mst, u, v):
            mst.add_edge(u, v, weight=weight)
    return mst
G = {
    "A": {"B": 2, "C": 3, "D": 3},
    "B": {"A": 2, "C": 4, "E": 3},
    "D": {"A": 3, "C": 5, "F": 5},
    "C": {"A": 3, "B": 4, "D": 5, "E": 1, "F": 6},
    "E": {"B": 3, "C": 1, "F": 8},
    "F": {"C": 6, "D": 7, "E": 8, "G": 9},
    "G": {"F": 9}
    }
w = G 
MST = kruskal(G, w)
print("Minimum Spanning Tree Edges:", MST.edges(data=True))

Minimum Spanning Tree Edges: [('A', 'B', {'weight': 2}), ('A', 'C', {'weight': 3}), ('A', 'D', {'weight': 3}), ('D', 'F', {'weight': 5}), ('C', 'E', {'weight': 1}), ('F', 'G', {'weight': 9})]


## Prim's Algorithm

In [3]:
from heapq import heappop, heappush,heapify
def prim(graph):
    mst = []
    visited = set()
    start_node = next(iter(graph))
    visited.add(start_node)
    queue = [(weight, start_node, v) for v, weight in graph[start_node].items()]
    heapify(queue)
    while queue:
        weight, u, v = heappop(queue)
        if v not in visited:
            visited.add(v)
            mst.append((u, v, weight))
            for w, weight in graph[v].items():
                if w not in visited:
                    heappush(queue, (weight, v, w))
    return mst
# Test the Prim’s algorithm
graph = {
    "A": {"B": 2, "C": 3, "D": 3},
    "B": {"A": 2, "C": 4, "E": 3},
    "D": {"A": 3, "C": 5, "F": 5},
    "C": {"A": 3, "B": 4, "D": 5, "E": 1, "F": 6},
    "E": {"B": 3, "C": 1, "F": 8},
    "F": {"C": 6, "D": 7, "E": 8, "G": 9},
    "G": {"F": 9}
    }
print(prim(graph))

[('A', 'B', 2), ('A', 'C', 3), ('C', 'E', 1), ('A', 'D', 3), ('D', 'F', 5), ('F', 'G', 9)]


## Prim's Algorithm validation with another graph

In [51]:
from heapq import heappop, heappush,heapify
def prim(graph):
    mst = []
    visited = set()
    start_node = next(iter(graph))
    visited.add(start_node)
    queue = [(weight, start_node, v) for v, weight in graph[start_node].items()]
    heapify(queue)
    while queue:
        weight, u, v = heappop(queue)
        if v not in visited:
            visited.add(v)
            mst.append((u, v, weight))
            for w, weight in graph[v].items():
                if w not in visited:
                    heappush(queue, (weight, v, w))
    return mst
# Test the Prim’s algorithm
graph = {
    "G": {"B": 5, "A": 4},
    "B": {"G": 5, "A": 1, "E": 2, "D": 2},
    "A": {"G": 4, "B": 1, "D": 4},
    "D": {"A": 4, "B": 2, "E": 3, "F": 5},
    "E": {"B": 2, "D": 3, "F": 1, "C": 4},
    "F": {"D": 5, "E": 1, "C": 3},
    "C": {"E": 4, "F": 3}
}
print(prim(graph))

[('G', 'A', 4), ('A', 'B', 1), ('B', 'D', 2), ('B', 'E', 2), ('E', 'F', 1), ('F', 'C', 3)]


## Dijkstra's Algorithm for undirected graph

In [7]:
import sys
from heapq import heapify, heappush, heappop

def Dijkstra(graph, src, dest):
    inf = sys.maxsize
    d = {node: {'distance': inf, 'pred': []} for node in graph}
    d[src]['distance'] = 0
    
    visited = []
    queue = [(0, src)]
    heapify(queue)
    
    while queue:
        distance, vertex = heappop(queue)
        if vertex not in visited:
            visited.append(vertex)
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    new_distance = d[vertex]['distance'] + graph[vertex][neighbor]
                    if new_distance < d[neighbor]['distance']:
                        d[neighbor]['distance'] = new_distance
                        d[neighbor]['pred'] = d[vertex]['pred'] + [vertex]
                        heappush(queue, (d[neighbor]['distance'], neighbor))
    
    print("Shortest Distance: " + str(d[dest]['distance']))
    print("Shortest Path: " + ' -> '.join(d[dest]['pred'] + [dest]))

if __name__ == "__main__":
    graph = {
        'A': {'B': 2, 'D': 8},
        'B': {'A': 2, 'D': 5, 'E': 6},
        'C': {'E': 8, 'F': 3},
        'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
        'E': {'B': 6, 'D': 3, 'C': 8, 'F': 1},
        'F': {'D': 2, 'E': 1, 'C': 3}
    }

    source = 'A'
    destination = 'D'
    Dijkstra(graph, source, destination)

Shortest Distance: 7
Shortest Path: A -> B -> D


## Dijkstra's Algorithm for undirected graph validation

In [61]:
import sys
from heapq import heapify, heappush, heappop

def Dijkstra(graph, src, dest):
    inf = sys.maxsize
    d = {node: {'distance': inf, 'pred': []} for node in graph}
    d[src]['distance'] = 0
    
    visited = []
    queue = [(0, src)]
    heapify(queue)
    
    while queue:
        distance, vertex = heappop(queue)
        if vertex not in visited:
            visited.append(vertex)
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    new_distance = d[vertex]['distance'] + graph[vertex][neighbor]
                    if new_distance < d[neighbor]['distance']:
                        d[neighbor]['distance'] = new_distance
                        d[neighbor]['pred'] = d[vertex]['pred'] + [vertex]
                        heappush(queue, (d[neighbor]['distance'], neighbor))
    
    print("Shortest Distance: " + str(d[dest]['distance']))
    print("Shortest Path: " + ' -> '.join(d[dest]['pred'] + [dest]))

if __name__ == "__main__":
    graph = {
    "G": {"B": 5, "A": 4},
    "B": {"G": 5, "A": 1, "E": 2, "D": 2},
    "A": {"G": 4, "B": 1, "D": 4},
    "D": {"A": 4, "B": 2, "E": 3, "F": 5},
    "E": {"B": 2, "D": 3, "F": 1, "C": 4},
    "F": {"D": 5, "E": 1, "C": 3},
    "C": {"E": 4, "F": 3}
}

    source = 'G'
    destination = 'C'
    Dijkstra(graph, source, destination)

Shortest Distance: 11
Shortest Path: G -> B -> E -> C


## Dijkstra Algorithm  with directed graph graph

In [69]:
import sys
from heapq import heapify, heappush, heappop

def Dijkstra(graph, src, dest):
    inf = sys.maxsize
    d = {node: {'distance': inf, 'pred': []} for node in graph}
    d[src]['distance'] = 0
    
    visited = []
    queue = [(0, src)]
    heapify(queue)
    
    while queue:
        distance, vertex = heappop(queue)
        if vertex not in visited:
            visited.append(vertex)
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    new_distance = d[vertex]['distance'] + graph[vertex][neighbor]
                    if new_distance < d[neighbor]['distance']:
                        d[neighbor]['distance'] = new_distance
                        d[neighbor]['pred'] = d[vertex]['pred'] + [vertex]
                        heappush(queue, (d[neighbor]['distance'], neighbor))
    
    print("Shortest Distance: " + str(d[dest]['distance']))
    print("Shortest Path: " + ' -> '.join(d[dest]['pred'] + [dest]))

if __name__ == "__main__":
    graph = {
    "A": {"B": 10, "C": 5},
    "B": {"D": 1},
    "C": {"B": 3, "E": 2},
    "D": {},
    "E": {"A": 2, "D": 6}
}
    source = 'A'
    destination = 'E'
    Dijkstra(graph, source, destination)

Shortest Distance: 7
Shortest Path: A -> C -> E


## Dijkstra Algorithm with directed graph graph validation

In [74]:
import sys
from heapq import heapify, heappush, heappop

def Dijkstra(graph, src, dest):
    inf = sys.maxsize
    d = {node: {'distance': inf, 'pred': []} for node in graph}
    d[src]['distance'] = 0
    
    visited = []
    queue = [(0, src)]
    heapify(queue)
    
    while queue:
        distance, vertex = heappop(queue)
        if vertex not in visited:
            visited.append(vertex)
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    new_distance = d[vertex]['distance'] + graph[vertex][neighbor]
                    if new_distance < d[neighbor]['distance']:
                        d[neighbor]['distance'] = new_distance
                        d[neighbor]['pred'] = d[vertex]['pred'] + [vertex]
                        heappush(queue, (d[neighbor]['distance'], neighbor))
    
    print("Shortest Distance: " + str(d[dest]['distance']))
    print("Shortest Path: " + ' -> '.join(d[dest]['pred'] + [dest]))

if __name__ == "__main__":
    graph = {
    "A": {"B": 10, "C": 4},
    "B": {"D": 5},
    "C": {"D": 9, "E": 6},
    "D": {"F": 8},
    "E": {"F": 3, "D":6},
    "F": {}
}
    source = 'A'
    destination = 'F'
    Dijkstra(graph, source, destination)

Shortest Distance: 13
Shortest Path: A -> C -> E -> F


## Bellman-Ford Algorithm

In [8]:
def bellman_ford(graph, source):
    distances = {vertex: float('inf') for vertex in graph}
    distances[source] = 0

    for _ in range(len(graph) - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

    for u in graph:
        for v, weight in graph[u].items():
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                raise ValueError("Graph contains negative weight cycle")

    return distances

graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'C': -2, 'E': 2},
    'C': {'E': 3},
    'D': {'B': -3},
    'E': {'D': 4}
}
source = 'A'

shortest_distances = bellman_ford(graph, source)
print(shortest_distances)

{'A': 0, 'B': 1, 'C': -1, 'D': 6, 'E': 2}


## Bellman-Ford Algorithm validation

In [87]:
def bellman_ford(graph, source):
    distances = {vertex: float('inf') for vertex in graph}
    distances[source] = 0

    for _ in range(len(graph) - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

    for u in graph:
        for v, weight in graph[u].items():
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                raise ValueError("Graph contains negative weight cycle")

    return distances

graph = {
    'A': {'B': 6, 'C': -5, 'E': 7},
    'B': {'D': -4, 'E': 3},
    'C': {'F': -3},
    'D': {'F': 10},
    'E': {'D': 9, 'C': 2},
    'F': {}
}
source = 'A'

shortest_distances = bellman_ford(graph, source)
print(shortest_distances)

{'A': 0, 'B': 6, 'C': -5, 'D': 2, 'E': 7, 'F': -8}


## Bellman-Ford Algorithm with negative cycle

In [86]:
def bellman_ford(graph, source):
    distances = {vertex: float('inf') for vertex in graph}
    distances[source] = 0

    for _ in range(len(graph) - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

    for u in graph:
        for v, weight in graph[u].items():
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                raise ValueError("Graph contains negative weight cycle")

    return distances

graph = {
    'A': {'C': 3},
    'B': {'A': 4, 'C': 7},
    'C': {'D': 5},
    'D': {'B': -15}
}
source = 'A'

shortest_distances = bellman_ford(graph, source)
print(shortest_distances)

ValueError: Graph contains negative weight cycle

## Bellman-Ford Algorithm with negative cycle validation

In [30]:
def bellman_ford(graph, source):
    distances = {vertex: float('inf') for vertex in graph}
    distances[source] = 0

    for _ in range(len(graph) - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

    for u in graph:
        for v, weight in graph[u].items():
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                raise ValueError("Graph contains negative weight cycle")

    return distances

graph = {
    'A': {'B': -2, 'C': 5},
    'B': {'D': 3},
    'C': {'B': 6, 'E': -7},
    'D': {'F': 10, 'C': -4},
    'E': {'D': 1, 'F': 5},
    'F': {}
}
source = 'A'

shortest_distances = bellman_ford(graph, source)
print(shortest_distances)

ValueError: Graph contains negative weight cycle